Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Descent Env"

In [48]:
import numpy as np
from descent_env import DescentEnv
import random 

In [49]:
# Cambiar render_mode a rgb_array para entrenar/testear
env = DescentEnv(render_mode='rgb_array')

Reading config from /Users/agustinvarela/bluesky/settings.cfg
Reading magnetic variation data
Loading global navigation database...
Reading cache: /Users/agustinvarela/bluesky/cache/navdata.p
Attempt to reimplement AREA from <bound method Area.set_area of <bluesky.plugins.area.Area object at 0x1739d0340>> to <bound method Area.set_area of <bluesky.plugins.area.Area object at 0x1739d0340>>
Attempt to reimplement EXP from <function init_plugin.<locals>.<lambda> at 0x107a62950> to <function init_plugin.<locals>.<lambda> at 0x1739c1bd0>
Attempt to reimplement TAXI from <bound method Area.set_taxi of <bluesky.plugins.area.Area object at 0x1739d0340>> to <bound method Area.set_taxi of <bluesky.plugins.area.Area object at 0x1739d0340>>
Successfully loaded plugin AREA
Attempt to reimplement DATAFEED from <bound method Modesbeast.toggle of <bluesky.plugins.adsbfeed.Modesbeast object at 0x1739d3070>> to <bound method Modesbeast.toggle of <bluesky.plugins.adsbfeed.Modesbeast object at 0x34fd801f0

Observation Space

In [50]:
env.observation_space

Dict('altitude': Box(-inf, inf, (1,), float64), 'runway_distance': Box(-inf, inf, (1,), float64), 'target_altitude': Box(-inf, inf, (1,), float64), 'vz': Box(-inf, inf, (1,), float64))

Action Space

In [51]:
env.action_space

Box(-1.0, 1.0, (1,), float64)

Discretización de los estados

**Nota:** es importante que chequeen el espacio de observación y el espacio de acción del entorno. Los números usados son ejemplos y pueden no ser correctos

In [52]:
altitude_space = np.linspace(0, 2.5, 50)  
vertical_velocity_space = np.linspace(-3, 3, 100)
target_altitude_space = np.linspace(0, 1, 50) 
runway_distance_space = np.linspace(-1, 1, 50)  


#print("altitude_space:", altitude_space)
#print("vertical_velocity_space:", vertical_velocity_space)
#print("target_altitude_space:", target_altitude_space)
#print("runway_distance_space:", runway_distance_space)

"""
import gymnasium as gym
from descent_env import DescentEnv
import numpy as np
import random

env = DescentEnv(render_mode="human") 
obs, info = env.reset()

num_episodes = 5 
max_steps_per_episode = 500 

all_altitudes = []
all_vertical_velocities = []
all_target_altitudes = []
all_runway_distances = []

for episode in range(num_episodes):
    obs, info = env.reset()
    done = False
    steps = 0
    print(f"\n--- Episodio {episode + 1} ---")
    while not done and steps < max_steps_per_episode:
        # Aquí generas una acción aleatoria válida para tu entorno
        actions = [-1.0, -0.5, 0.0, 0.5, 1.0] # Acciones discretas sugeridas en el notebook
        action = np.array([random.choice(actions)])

        obs, reward, done, truncated, info = env.step(action)

        # Recolectar datos
        all_altitudes.append(obs["altitude"][0])
        all_vertical_velocities.append(obs["vz"][0])
        all_target_altitudes.append(obs["target_altitude"][0])
        all_runway_distances.append(obs["runway_distance"][0])
        steps += 1
        
    env.render()
    env.close()

# Analizar los datos recolectados
print("\n--- Análisis de rangos observados ---")
print(f"Altitud: Min={np.min(all_altitudes):.2f}, Max={np.max(all_altitudes):.2f}")
print(f"Velocidad Vertical: Min={np.min(all_vertical_velocities):.2f}, Max={np.max(all_vertical_velocities):.2f}")
print(f"Altitud Objetivo: Min={np.min(all_target_altitudes):.2f}, Max={np.max(all_target_altitudes):.2f}")
print(f"Distancia Pista: Min={np.min(all_runway_distances):.2f}, Max={np.max(all_runway_distances):.2f}")
"""

'\nimport gymnasium as gym\nfrom descent_env import DescentEnv\nimport numpy as np\nimport random\n\nenv = DescentEnv(render_mode="human") \nobs, info = env.reset()\n\nnum_episodes = 5 \nmax_steps_per_episode = 500 \n\nall_altitudes = []\nall_vertical_velocities = []\nall_target_altitudes = []\nall_runway_distances = []\n\nfor episode in range(num_episodes):\n    obs, info = env.reset()\n    done = False\n    steps = 0\n    print(f"\n--- Episodio {episode + 1} ---")\n    while not done and steps < max_steps_per_episode:\n        # Aquí generas una acción aleatoria válida para tu entorno\n        actions = [-1.0, -0.5, 0.0, 0.5, 1.0] # Acciones discretas sugeridas en el notebook\n        action = np.array([random.choice(actions)])\n\n        obs, reward, done, truncated, info = env.step(action)\n\n        # Recolectar datos\n        all_altitudes.append(obs["altitude"][0])\n        all_vertical_velocities.append(obs["vz"][0])\n        all_target_altitudes.append(obs["target_altitude"][0

Obtener el estado a partir de la observación

In [53]:
def get_state(obs):
    alt = obs['altitude'][0]
    vz = obs['vz'][0]
    target_alt = obs['target_altitude'][0]
    runway_dist = obs['runway_distance'][0]
    alt_idx = np.digitize(alt, altitude_space)
    vz_idx = np.digitize(vz, vertical_velocity_space)
    target_alt_idx = np.digitize(target_alt, target_altitude_space)
    runway_dist_idx = np.digitize(runway_dist, runway_distance_space)
    return alt_idx, vz_idx, target_alt_idx, runway_dist_idx

In [26]:
obs = env.observation_space.sample()
print(obs)
state = get_state(obs) # Ejemplo de obs
state

OrderedDict([('altitude', array([-0.53871572])), ('runway_distance', array([-0.6354918])), ('target_altitude', array([1.19319792])), ('vz', array([-1.98032329]))])


(0, 17, 50, 9)

Discretización de las acciones

In [27]:
actions = list(np.linspace(-1, 1, 10))
actions

[-1.0,
 -0.7777777777777778,
 -0.5555555555555556,
 -0.33333333333333337,
 -0.11111111111111116,
 0.11111111111111116,
 0.33333333333333326,
 0.5555555555555554,
 0.7777777777777777,
 1.0]

In [28]:
def get_sample_action():
    return random.choice(actions)

Inicilización de la tabla Q

In [29]:
Q = np.zeros((len(altitude_space), len(vertical_velocity_space), len(target_altitude_space), len(runway_distance_space), len(actions)))
Q

array([[[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0.

Obtención de la acción a partir de la tabla Q

In [30]:
def optimal_policy(state, Q):
    action = actions[np.argmax(Q[state])]
    return action

Epsilon-Greedy Policy

In [31]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = get_sample_action()
    else:
        action = optimal_policy(state, Q)
        
    return action

Ejemplo de episodio 

In [47]:
obs,_ = env.reset()
print(obs)
done = False
total_reward = 0
state = get_state(obs)
steps = 0
while not done:
    steps += 1
    # Acción del modelo
    action = epsilon_greedy_policy(state, Q, 0.5)
    
    # Indice de la accion en Q
    action_idx = actions.index(action)
    
    # Acción del ambiente
    real_action = np.array([action])
     
    obs, reward, done, _, _ = env.step(real_action)
    next_state = get_state(obs)
    
    alpha = 0.1
    gamma = 0.99
   # Usar action_idx para actualizar Q
    Q[state][action_idx] = Q[state][action_idx] + alpha * (reward + gamma * np.max(Q[next_state]) - Q[state][action_idx])

   # Actualizar estado
    state = next_state
   
    total_reward += reward

    env.render()

env.close()    
print('total_reward', total_reward)
print('steps', steps)

{'altitude': array([0.225]), 'vz': array([0.]), 'target_altitude': array([0.36]), 'runway_distance': array([0.5])}
total_reward -147.19534777166666
steps 22
